In [17]:
from collections import defaultdict
from functools import lru_cache
import itertools as it
import numpy as np

from adict import adict

In [21]:
@lru_cache(None)
def x(indices):
    """Variable label"""
    i, j = indices
    return f"x{i}_{j}"

@lru_cache(None)
def ind(label: str):
    """Indices from variable"""
    i, j = label[1:].split('_')
    return (int(i), int(j))

def row_regions(N):
    return [[(i, j) for j in range(N)] for i in range(N)]

def column_regions(N):
    return [[(i, j) for i in range(N)] for j in range(N)]

def region_lists(grid: np.array):
    """Lists of region indices from grid"""
    regions = defaultdict(list)
    for (i, j), r in np.ndenumerate(grid):
        regions[r].append((i, j))
    return list(regions.values())

In [9]:
"""Penalties and constraints"""

def penalties(N):
    """Closest neighbours should be different.
    This is a penalty of type x * y, which is 1 iff x = y = 1."""

    Q = adict(int)
    positions = it.product(range(N-1), repeat=2)  # Exclude last row/column

    for (i, j) in positions:
        neighbours = [(i+1, j), (1, j+1), (i+1, j+1)]
        for (k, l) in neighbours:
            Q[x((i,j)), x((k,l))] += 1
        Q[x((i+1, j)), x((i, j+1))]  # Anti-diagonal

    for l in range(N-1):
        Q[x((N-1, l  )), x((N-1, l+1))] += 1
        Q[x((l,   N-1)), x((l+1, N-1))] += 1
        
    return Q

def region_constraint(region: list, stars: int):
    """All points in :region: sum to :stars:."""
    Q = adict(int)
    for (i, j) in region:
        Q[ ( x((i,j)) , x((i,j)) ) ] += (-stars+1)
        for (l, p) in region:
            if (l, p) != (i, j):
                Q[ ( x((i,j)) , x((l,p)) ) ] += 1
    return Q

In [40]:
# Input data

N = 5  # Grid size
nstars = 1  # Number of stars

grid = np.array([[0,1,1,1,1],
                [0,1,1,1,2],
                [0,0,0,2,2],
                [3,0,3,2,2],
                [3,3,3,4,4]])

star = np.array([[0,1,0,0,0],
                [0,0,0,0,1],
                [0,0,1,0,0],
                [1,0,0,0,0],
                [0,0,0,1,0]])

In [41]:
# Build QUBO

Q = penalties(N)

regions = it.chain(
    region_lists(grid),
    row_regions(N),
    column_regions(N),
)

for region in regions:
    Q += region_constraint(region, nstars)

In [12]:
### Annealing ###

In [42]:
# Create Sampler

import neal

sampler = neal.SimulatedAnnealingSampler()

In [43]:
# Try Sampling

sampleset = sampler.sample_qubo(Q, num_reads=20)
print(sampleset)

   x0_0 x0_1 x0_2 x0_3 x0_4 x1_0 x1_1 x1_2 x1_3 x1_4 ... x4_4 energy num_oc.
0     0    0    1    0    0    0    0    0    0    0 ...    0    0.0       1
1     0    1    0    0    0    0    0    0    0    0 ...    1    0.0       1
2     0    0    1    0    0    0    0    0    0    1 ...    0    0.0       1
3     0    1    0    0    0    1    0    0    0    0 ...    0    0.0       1
4     0    1    0    0    0    0    0    0    0    0 ...    0    0.0       1
5     0    1    0    0    0    0    0    0    0    0 ...    1    0.0       1
6     0    0    0    0    0    0    0    0    0    1 ...    0    0.0       1
7     0    1    0    0    0    1    0    0    0    0 ...    0    0.0       1
8     0    0    0    0    0    0    0    1    0    0 ...    0    0.0       1
9     0    0    0    0    0    0    0    0    0    1 ...    0    0.0       1
10    0    0    0    0    1    0    0    0    0    0 ...    0    0.0       1
11    0    0    0    0    0    0    0    0    0    0 ...    0    0.0       1

In [44]:
sample = sampler.sample_qubo(Q).first.sample
sample

{'x0_0': 0,
 'x0_1': 0,
 'x0_2': 0,
 'x0_3': 0,
 'x0_4': 0,
 'x1_0': 0,
 'x1_1': 0,
 'x1_2': 1,
 'x1_3': 0,
 'x1_4': 0,
 'x2_0': 0,
 'x2_1': 0,
 'x2_2': 0,
 'x2_3': 0,
 'x2_4': 0,
 'x3_0': 0,
 'x3_1': 0,
 'x3_2': 0,
 'x3_3': 0,
 'x3_4': 1,
 'x4_0': 0,
 'x4_1': 0,
 'x4_2': 0,
 'x4_3': 1,
 'x4_4': 0}

In [55]:
def region_criterion(sample, region: list, stars: int):
    '''Check if :sample: respects :region: criterion'''
    region_sum = 0
    for (i,j) in region:
        var = 'x_{}_{}'.format(i,j)
        region_sum += sample(var)
    if region_sum == 2:
        return True
    else: 
        return False
    
def proximity_criterion(sample):
    star_positions = [ind(var) for var, value in sample.items() if value == 1]
    return all(
        abs(i - i_) > 1 and abs(j - j_) > 1
        for (i, j), (i_, j_) in it.combinations(star_positions, 2)
    )

def confirm_solution(sample, regions, stars):
    '''Check if sample corresponds solves problem'''
    
    for region in regions:
        if region_criterion(sample, region, starts) == False:
            return False
        
    if proximity_criterion(sample, starts) == False:
            return False
    
    return True

In [46]:
star_positions = [ind(var) for var, value in sample.items() if value == 1]
solution = np.zeros((N, N), dtype=int)
for (i, j) in star_positions:
    solution[i, j] = 1
solution

array([[0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0]])

In [47]:
proximity_criterion(sample)

False

In [58]:
for sample in sampleset:
    #if proximity_criterion(sample):
        star_positions = [ind(var) for var, value in sample.items() if value == 1]
        print(star_positions)
        solution = np.zeros((N, N), dtype=int)
        for (i, j) in star_positions:
            solution[i, j] = 1
        print(solution)

[(0, 2), (2, 3), (4, 0)]
[[0 0 1 0 0]
 [0 0 0 0 0]
 [0 0 0 1 0]
 [0 0 0 0 0]
 [1 0 0 0 0]]
[(1, 3), (3, 0)]
[[0 0 0 0 0]
 [0 0 0 1 0]
 [0 0 0 0 0]
 [1 0 0 0 0]
 [0 0 0 0 0]]
[(1, 1), (4, 3)]
[[0 0 0 0 0]
 [0 1 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 1 0]]
[(0, 3), (4, 2)]
[[0 0 0 1 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 1 0 0]]
[(0, 2), (1, 4), (3, 0), (4, 3)]
[[0 0 1 0 0]
 [0 0 0 0 1]
 [0 0 0 0 0]
 [1 0 0 0 0]
 [0 0 0 1 0]]
[(0, 4), (4, 0)]
[[0 0 0 0 1]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [1 0 0 0 0]]
[(0, 0), (1, 2), (4, 1)]
[[1 0 0 0 0]
 [0 0 1 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 1 0 0 0]]
[(3, 1), (4, 0)]
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]]
[(0, 4), (2, 3)]
[[0 0 0 0 1]
 [0 0 0 0 0]
 [0 0 0 1 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[(1, 4), (2, 2), (3, 0)]
[[0 0 0 0 0]
 [0 0 0 0 1]
 [0 0 1 0 0]
 [1 0 0 0 0]
 [0 0 0 0 0]]
[(1, 2), (3, 4), (4, 1)]
[[0 0 0 0 0]
 [0 0 1 0 0]
 [0 0 0 0 0]
 [0 0 0 0 1]
 [0 1 0 0 0]]
[(0, 1), (1, 0), (4, 2)]
[[0 1 0 0 0]
 